Description on what is done in this notebook
To be filled later

In [1]:
# For using cuda GPU backend
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
# Initializing default parameters
path = "data/ml-small/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

Let us know explore the data and see .

In [4]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
len(ratings)

100004

In [6]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

#### We update the movie and user ids so that they are contiguous integers, which we want when using embeddings.

In [7]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [8]:
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])
ratings.movieId = ratings.movieId.apply(lambda x : movieid2idx[x])

In [9]:
ratings.head()

,userId,movieId,rating,timestamp
0,0,0,2.5,1260759144
1,0,1,3.0,1260759179
2,0,2,3.0,1260759182
3,0,3,2.0,1260759185
4,0,4,4.0,1260759205


In [10]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), 
    ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())
user_min, user_max, movie_min, movie_max

(0, 670, 0, 9065)

In [11]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(671, 9066)

In [12]:
# Let us decide on number of latent factors we''ll use
n_factors = 50

In [13]:
# Splitting training and validation set
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

In [14]:
len(val)

19804

## basic model : Dot product of user and movie embeddings

In [15]:
user_in = Input(shape=(1,),dtype = 'int64', name = 'user_in')
movie_in = Input(shape = (1,), dtype = 'int64', name = 'movie_in')

In [16]:
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

In [17]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [18]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=1, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80200 samples, validate on 19804 samples
Epoch 1/1
80200/80200 [==============================] - 6s - loss: 10.0040 - val_loss: 3.9107


In [19]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=3, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80200 samples, validate on 19804 samples
Epoch 1/3
80200/80200 [==============================] - 6s - loss: 3.1529 - val_loss: 2.0476
Epoch 2/3
80200/80200 [==============================] - 6s - loss: 2.4166 - val_loss: 1.6759
Epoch 3/3
80200/80200 [==============================] - 6s - loss: 2.2450 - val_loss: 1.5403


In [20]:
model.optimizer.lr=0.001

In [21]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80200 samples, validate on 19804 samples
Epoch 1/6
80200/80200 [==============================] - 6s - loss: 2.1733 - val_loss: 1.4927
Epoch 2/6
80200/80200 [==============================] - 6s - loss: 2.1316 - val_loss: 1.4705
Epoch 3/6
80200/80200 [==============================] - 6s - loss: 2.1033 - val_loss: 1.4639
Epoch 4/6
80200/80200 [==============================] - 6s - loss: 2.0798 - val_loss: 1.4572
Epoch 5/6
80200/80200 [==============================] - 6s - loss: 2.0599 - val_loss: 1.4545
Epoch 6/6
80200/80200 [==============================] - 6s - loss: 2.0383 - val_loss: 1.4544


In [22]:
## Learning rate seems to be too slow

In [23]:
model.optimizer.lr=0.01

In [24]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=1, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80200 samples, validate on 19804 samples
Epoch 1/1
80200/80200 [==============================] - 6s - loss: 2.0193 - val_loss: 1.4618


#### The best benchmarks are a bit over 0.9, so this model doesn't seem to be working that well...

### Bias
Previously we didnt add bias terms which is very crucial. Let us try with bias

In [25]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [26]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [27]:
def create_bias(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

In [28]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [29]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, mb], mode='sum')
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [30]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=1, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80200 samples, validate on 19804 samples
Epoch 1/1
80200/80200 [==============================] - 6s - loss: 8.7196 - val_loss: 3.1173


In [31]:
model.optimizer.lr=0.01

In [32]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=4, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80200 samples, validate on 19804 samples
Epoch 1/4
80200/80200 [==============================] - 6s - loss: 2.5648 - val_loss: 1.6259
Epoch 2/4
80200/80200 [==============================] - 6s - loss: 1.9740 - val_loss: 1.3316
Epoch 3/4
80200/80200 [==============================] - 6s - loss: 1.8138 - val_loss: 1.2331
Epoch 4/4
80200/80200 [==============================] - 6s - loss: 1.7210 - val_loss: 1.1703


In [33]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=2, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80200 samples, validate on 19804 samples
Epoch 1/2
80200/80200 [==============================] - 6s - loss: 1.6448 - val_loss: 1.1386
Epoch 2/2
80200/80200 [==============================] - 6s - loss: 1.5742 - val_loss: 1.1084


In [34]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=2, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80200 samples, validate on 19804 samples
Epoch 1/2
80200/80200 [==============================] - 6s - loss: 1.5052 - val_loss: 1.0859
Epoch 2/2
80200/80200 [==============================] - 6s - loss: 1.4395 - val_loss: 1.0574


In [35]:
model.optimizer.lr=0.001

In [36]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=10, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80200 samples, validate on 19804 samples
Epoch 1/10
80200/80200 [==============================] - 6s - loss: 1.3732 - val_loss: 1.0304
Epoch 2/10
80200/80200 [==============================] - 6s - loss: 1.3088 - val_loss: 1.0119
Epoch 3/10
80200/80200 [==============================] - 6s - loss: 1.2457 - val_loss: 0.9896
Epoch 4/10
80200/80200 [==============================] - 6s - loss: 1.1852 - val_loss: 0.9649
Epoch 5/10
80200/80200 [==============================] - 6s - loss: 1.1265 - val_loss: 0.9470
Epoch 6/10
80200/80200 [==============================] - 6s - loss: 1.0716 - val_loss: 0.9291
Epoch 7/10
80200/80200 [==============================] - 6s - loss: 1.0189 - val_loss: 0.9118
Epoch 8/10
80200/80200 [==============================] - 6s - loss: 0.9695 - val_loss: 0.8945
Epoch 9/10
80200/80200 [==============================] - 6s - loss: 0.9226 - val_loss: 0.8789
Epoch 10/10
80200/80200 [==============================] - 6s - loss: 0.8792 - val_loss: 0.8657

In [37]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=5, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80200 samples, validate on 19804 samples
Epoch 1/5
80200/80200 [==============================] - 6s - loss: 0.8385 - val_loss: 0.8519
Epoch 2/5
80200/80200 [==============================] - 6s - loss: 0.8010 - val_loss: 0.8439
Epoch 3/5
80200/80200 [==============================] - 6s - loss: 0.7665 - val_loss: 0.8323
Epoch 4/5
80200/80200 [==============================] - 6s - loss: 0.7349 - val_loss: 0.8244
Epoch 5/5
80200/80200 [==============================] - 6s - loss: 0.7070 - val_loss: 0.8174


In [40]:
model.save_weights(model_path + 'InitialBias.h5')

In [41]:
model.load_weights(model_path + 'InitialBias.h5')

In [42]:
model.predict([np.array([3]), np.array([6])])

array([[ 4.8666]], dtype=float32)

### Analysis

### Neural Net

Until now we just calculated a dot product and bias terms. Let us try wiht one hidden layer

In [45]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [46]:
x = merge([u, m], mode='concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')

In [47]:
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=8, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80200 samples, validate on 19804 samples
Epoch 1/8
80200/80200 [==============================] - 7s - loss: 2.4593 - val_loss: 0.9039
Epoch 2/8
80200/80200 [==============================] - 7s - loss: 1.4458 - val_loss: 0.8544
Epoch 3/8
80200/80200 [==============================] - 7s - loss: 1.2152 - val_loss: 0.8283
Epoch 4/8
80200/80200 [==============================] - 7s - loss: 1.0382 - val_loss: 0.8169
Epoch 5/8
80200/80200 [==============================] - 7s - loss: 0.9108 - val_loss: 0.8048
Epoch 6/8
80200/80200 [==============================] - 7s - loss: 0.8461 - val_loss: 0.7879
Epoch 7/8
80200/80200 [==============================] - 7s - loss: 0.8206 - val_loss: 0.7846
Epoch 8/8
80200/80200 [==============================] - 7s - loss: 0.8093 - val_loss: 0.7848


In [48]:
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=2, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80200 samples, validate on 19804 samples
Epoch 1/2
80200/80200 [==============================] - 7s - loss: 0.8042 - val_loss: 0.7880
Epoch 2/2
80200/80200 [==============================] - 7s - loss: 0.8016 - val_loss: 0.7867


## Seems like we are underfitting, let use decrease the droupout

In [50]:
x = merge([u, m], mode='concat')
x = Flatten()(x)
x = Dropout(0.25)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.70)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')

In [51]:
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=8, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80200 samples, validate on 19804 samples
Epoch 1/8
80200/80200 [==============================] - 7s - loss: 2.1023 - val_loss: 0.8648
Epoch 2/8
80200/80200 [==============================] - 7s - loss: 1.2864 - val_loss: 0.8114
Epoch 3/8
80200/80200 [==============================] - 7s - loss: 1.1352 - val_loss: 0.8097
Epoch 4/8
80200/80200 [==============================] - 7s - loss: 0.9996 - val_loss: 0.7947
Epoch 5/8
80200/80200 [==============================] - 7s - loss: 0.9079 - val_loss: 0.7969
Epoch 6/8
80200/80200 [==============================] - 7s - loss: 0.8473 - val_loss: 0.7937
Epoch 7/8
80200/80200 [==============================] - 7s - loss: 0.8117 - val_loss: 0.7902
Epoch 8/8
80200/80200 [==============================] - 7s - loss: 0.8002 - val_loss: 0.7887


In [52]:
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=3, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80200 samples, validate on 19804 samples
Epoch 1/3
80200/80200 [==============================] - 7s - loss: 0.7929 - val_loss: 0.7866
Epoch 2/3
80200/80200 [==============================] - 7s - loss: 0.7888 - val_loss: 0.7854
Epoch 3/3
80200/80200 [==============================] - 7s - loss: 0.7852 - val_loss: 0.7870
